In [1]:
import json
import logging
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import trange, tqdm
from time import sleep
import pymongo
import requests


In [2]:
client_db = pymongo.MongoClient(port=27017)
db = client_db.py_book
table = db['shahreketab']
df = pd.DataFrame(table.find({}, {
    '_id': 0
}))

In [3]:
text = df.text[0]
soup = BeautifulSoup(text)
cat = soup.select('ol.breadcrumb')[0].findAll("li", {"class": "breadcrumb-item"})[1]["title"]
title = soup.select('h1.ProductTitle')[0].text.strip()




In [4]:
for i, row in df.iterrows():
    if row.status_code == 200:
        soup = BeautifulSoup(row.text)
        cat = soup.select('ol.breadcrumb')[0].findAll("li", {"class": "breadcrumb-item"})[1]["title"].strip()
        df.at[i, "cat"] = cat
        if cat != 'کتاب':
            continue
        df.at[i, "title"] = soup.select('h1.ProductTitle')[0].text.strip()

IndexError: list index out of range

In [5]:
df[df['title'].isnull()]

,id,url,url_result,text,status_code,cat,title
478,394149,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,نوشت افزار,NaN
3453,391141,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,نشریات,NaN
3535,391056,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN
3536,391055,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN
3537,391054,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN
...,...,...,...,...,...,...,...
8258,392254,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN
8259,392246,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN
8260,391948,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN
8261,391581,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200,NaN,NaN


In [ ]:
df[['url','url_result','status_code','cat','title']]